## Save Model

In [1]:
save_model = False  # we have already saved the model
if save_model:
    from transformers import AutoTokenizer, AutoModel
    from pathlib import Path

    tokenizer = AutoTokenizer.from_pretrained('AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2')
    model = AutoModel.from_pretrained('AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2')

    sim_path = Path('/mnt/data/factcheck/tweets/similarity')

    tokenizer_path = Path(sim_path, 'models', 'mstsb_tokenizer')
    tokenizer.save_pretrained(tokenizer_path)

    model_path = Path(sim_path, 'models', 'mstsb_model')
    model.save_pretrained(model_path)

## Common Constants

In [2]:
import json
from pathlib import Path
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

home_dir = Path('/mnt/data/factcheck/tweets/similarity')
source_path = home_dir / 'data' / 'tweets_with_titles.jsonl'
tensor_path = home_dir / 'data' / 'pooled_tensors.pt'
norm_tensor_path = home_dir / 'data' / 'norm_pooled_tensors.pt'

def load_jsonl(path):
    with open(path) as f:
        return [json.loads(line) for line in f]

## Create Pooled Vectors

#### Load Model

In [5]:
load_and_tokenize = True  # set to False if tokens have been already created
                          # and you do not wish to regenerate them

if load_and_tokenize:

    from transformers import AutoTokenizer, AutoModel

    tokenizer_path = home_dir / 'models' / 'mstsb_tokenizer'
    model_path =     home_dir / 'models' / 'mstsb_model'

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModel.from_pretrained(model_path)
    model.to(device)
    model.eval()
    print('loaded')

loaded


#### Save Pooled Vectors

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


if load_and_tokenize:

    import json
    from tqdm.auto import tqdm
    from torch.utils.data import DataLoader, TensorDataset

    all_lines = load_jsonl(source_path)
    print(all_lines[0].keys())

    claims = [item['text'] for item in all_lines]
    encoded_claims = tokenizer(claims, padding=True, truncation=True, return_tensors='pt')
    print('Tokenized', encoded_claims.keys())

    ds = TensorDataset(encoded_claims['input_ids'], encoded_claims['attention_mask'])
    dl = DataLoader(dataset=ds, batch_size=1000, shuffle=False)

    tq = tqdm(range(len(dl)))

    res = None
    with torch.no_grad():
        for input_ids, attention_mask in dl:        
            tq.update(1)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            # Original computation:
            # out = model(input_ids=input_ids, attention_mask=attention_mask)['pooler_output'].to('cpu')
            # New computation:
            model_output = model(input_ids=input_ids, attention_mask=attention_mask)
            out = mean_pooling(model_output, attention_mask).to('cpu')
            
            res = out if res is None else torch.cat((res, out))

    print('Shape output:', res.shape, '  Len input:', len(all_lines))
    torch.save(res, tensor_path)

dict_keys(['id', 'text'])
Tokenized dict_keys(['input_ids', 'attention_mask'])


  0%|          | 0/186 [00:00<?, ?it/s]

## Load and Normalize Vectors

In [7]:
with torch.no_grad():
    vecs = torch.load(tensor_path).to(device)
    norms = vecs.pow(2).sum(dim=1, keepdims=True).sqrt()
    vecs = vecs / norms
    
print(vecs.device, norms.device)

all_lines = load_jsonl(source_path)

cpu cpu


In [10]:
print(all_lines[0])
print(len(all_lines))
print(vecs.shape)

{'id': 1422203548138450949, 'text': 'Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍'}
185161
torch.Size([185161, 768])


In [12]:
import re
import collections
import math
import random

def indices_containing(pattern_string, data):
    pat = re.compile(pattern_string, re.IGNORECASE)
    return [no for no, item in enumerate(data) if pat.search(item['text']) is not None]


def from_indices(indices, data, with_index=False):
    if with_index:
        return [(i, data[i]) for i in indices]
    return [data[i] for i in indices]


# def _top_closest_indices(tweet_nos, k, vectors):
#     with torch.no_grad():
#         # sel_vecs = vectors[tweet_nos]
#         sel_vec = vecs[tweet_nos][:,None]
#         mult = (vecs @ sel_vec).squeeze(dim=1)
#         values, indices = mult.topk(k)
#         return indices.tolist()
    
    
def _top_closest_indices(tweet_nos, k, vectors):
    with torch.no_grad():
        res = None
        batch_size = 3500000 // len(tweet_nos)
        batches = math.ceil(len(vecs)/batch_size)
        for i in range(batches):
            lo = i * batch_size
            hi = lo + batch_size # exclusive
            vecs_batch = vectors[lo:hi, :]         # n x 768
            vecs_resh = vecs_batch[:,None,:]    # n x 1 x 768

            sel = vecs[tweet_nos,:]       # k x 768
            sel_resh = sel.T[None, :, :]  #          -(T)-> 768 x k -[]-> 1 x 768 x k

            mult = vecs_resh.to(device) @ sel_resh.to(device)  # n x 1 x 768 @ 1 x 768 x k == n x 1 x k
            mult = mult.squeeze(dim=1).mean(dim=1, keepdims=True)  # n x 1 x k -> n x k -> n x 1
            res = mult if res is None else torch.vstack((res, mult))

        res = res.squeeze(dim=1)
        val, indices = res.topk(k)
        return indices.tolist()
    
    
def top_closest_indices(tweet_nos, k, vectors, data, excl=None):
    if isinstance(tweet_nos, int):
        tweet_nos = [tweet_nos]
    tweet_nos = [tweet_no] if isinstance(tweet_nos, int) else tweet_nos
    excl = set(excl) if excl is not None else []
    with torch.no_grad():
        multi = 4
        while True:
            tc_indices = _top_closest_indices(tweet_nos, multi * k, vectors)
            filtered_indices = [index for index in tc_indices if index not in excl]
            if len(filtered_indices) >= k  or  len(tc_indices) == len(data):
                break
            else:
                multi *= 4
        return filtered_indices[:k]
    
    
def pretty_print(tweet_list, with_nos = False):
    if with_nos:
        for no, twt in tweet_list:
            print(no, twt['text'], '\n')
    else:
        for twt in tweet_list:
            print(twt['text'], '\n')


def print_semantic_neighbors(search_pattern, data, k=10, max_num=2000):
    '''
    The function:
    (1) finds all tweets in the data that conform to the search pattern; and then
    (2) prints k tweets that are semanticaly closest to searched tweets, but do not conform 
    to the search pattern (ie are syntactically outside the search scope, but are semantically
    closest to the syntactic search result).
       (k tweets that are semantically closest = k tweets with the lowest sum of 
       cosine similarities to searched tweets).
    
    :param max_num: If more than max_num tweets are found in data in step (1), only max_num
    of such tweets (randomly chosen) are used for identifying semantically similar tweets in 
    step (2).
    Reason for the limitation: multiplying all tweets in the full dataset (186k) with too many 
    found tweets (> 10k for 'prezident') results in excessively long computation time).
    '''
    cont_indices_all = indices_containing(search_pattern, data)
    if len(cont_indices_all) > max_num:
        cont_indices_limit = random.sample(cont_indices_all, max_num)
    else:
        cont_indices_limit = cont_indices_all

    if len(cont_indices_all):
        tcf = top_closest_indices(cont_indices_limit, k=k, vectors=vecs, 
                                  data=all_lines, excl=cont_indices_all)
        filtered = from_indices(tcf, all_lines)
        pretty_print(filtered)
    else:
        print(f'No tweet with {search_pattern} found.')
        
        
def get_semantic_neighbors(search_pattern, data, k=10, max_num=2000):
    cont_indices_all = indices_containing(search_pattern, data)
    if len(cont_indices_all) > max_num:
        cont_indices_limit = random.sample(cont_indices_all, max_num)
    else:
        cont_indices_limit = cont_indices_all

    if len(cont_indices_all):
        tcf = top_closest_indices(cont_indices_limit, k=k, vectors=vecs,
                                  data=all_lines, excl=cont_indices_all)
        return tcf
    else:
        print(f'No tweet with {search_pattern} found.')
        
        
def get_twitter_ids_from_all_lines_indices(ids, all_lines):
    return [all_lines[idx]['id'] for idx in ids]

In [65]:
print_semantic_neighbors('ivermekt', all_lines, 5)
sn = get_semantic_neighbors('ivermekt', all_lines, 1000)
twitter_ids = get_twitter_ids_from_all_lines_indices(sn, all_lines)

idx = sn[0]
print(all_lines[idx])
print(len(twitter_ids))

➡️Oni už ten Ivemektin posoudili dávno jiní a tisíce lidí u nás a na Slovensku mu vděčí za život.  ➡️ A proto mu u nás Covidová  mafie neprodlouží povolení k užívání. 

 Reálná data z praxe.   [Funguje ivermectin? Brněnští lékaři zkoumali účinky experimentálního léku  |  Zdraví  |  Zprávy  |  Brněnská Drbna - zprávy z Brna a Jihomoravského kraje] 

My máme doma zkušenost jinou. Iva si vyzkoušela tvrdou práci zdravotní sestry jako dobrovolnice, takže až moc dobře víme, jak náročnou práci zdravotní sestry mají. Neskutečně. Dnes mají svůj Mezinárodní den. Tak prosím využijme každou příležitost a poděkujme jim. ❤ 

Jak lhal, lže a vždy lhát bude. Jsem rád, že KONEČNĚ používá studie, na které je měsíce upozorňujeme. Proč tajil lidem rok léčbu IV-C a Isopri., to se jej ještě budeme ptát. O léčbě IV-C jsem jej PROKAZATELNĚ informoval v březnu 2020 Video👇  [Použití této funkce vám bylo dočasně zablokováno.] 

1: Kdo dnes neléčí #ivermectine, přestože může a nebo jej aktivně neshání VRAŽDÍ!!!  

In [8]:
print_semantic_neighbors('očkován', all_lines, 5)

 Ano, proč ne? Víte co ještě nikdy nikdo neprovedl? Studii porovnávající výsledky cílené podpory imunity contra vakcinace. Na základě čeho tedy soudíme, že vakcinace je správná cesta? 

Je tady někdo překvapen? Informace o tom, že vakcinovaní nejsou imunní a mnohdy plní nemocnice po celém světě jsme tady zveřejňovali již před několika měsíci.   [V domově, kde zemřel nakažený senior, dostali covid další dva proočkovaní - Novinky.cz] 

Při všem respektu k utrpení blízkých, přátel i studentů chci vědět jestli byli vakcinovaní? Dnes není čas na falešnou ohleduplnost, žijeme v době nevídaného, nikdy dříve nepoznaného nátlaku na celou společnost a jsou informace, jejichž utajování se rovná masové vraždě:( ! 

Krátký report o počtu vakcín a dnešním jednání s hejtmany. Víc ve videu 🇨🇿   

Takže, když jsme informovali o “vakcinaci” lidí tak, jak se dnes vakcinuje dobytek, OPĚT jsme měli pravdu.  Pro některé “lidi” jste Vy a#i Vaše rodina, včetně vašich dětí, pouze dobytkem:(.  Pochopte to a BRA

In [67]:
print_semantic_neighbors('preziden', all_lines, 5)

Považuji za slušné, že své stanovisko k aktuálnímu politickému dění sdělím nejdříve panu premiérovi:  [Ubýváte. Přijdu se na vás podívat, vysmál se Miloš Zeman demonstrantům z Václaváku. A trousil perly k vykopnutí Rusů za Skripala, k Nikulinovi a k senátorům, kteří ho zvou „na výslech“ | ParlamentniListy.cz – politika ze všech stran] 

Ráno jsme společně s mým slovenským protějškem Andrejem Dankem položili květiny k bustě Františka Palackého. Teď už jsme ale zpátky u jednacího stolu, kde probíhá společné zasedání předsedů Poslanecké sněmovny ČR a Národní rady SK 🇨🇿🤝🇸🇰   

Můžeme třikrát hádat, komu budou lidé v členských zemích EU věřit : Právní službě Evropské komise, nebo českému premiérovi, mezinárodně proslulému svojí prolhaností? Pane premiére, přestaňte nám dělat ostudu v celé Evropě a odstupte!  [Babiš o svém střetu zájmů:  Je to načasovaná pseudokauza před evropskými volbami - Echo24.cz] 

Zajímavý rozhovor, šla k věci. Bavili jsme se o posilování českého vlivu v Evropě a o na

In [66]:
print_semantic_neighbors('uprchl', all_lines, 5)

EU nám nutí změnu azylového zákona. Musíme zavést institut dočasného strpění. Tedy nemožnost okamžitě vyhostit migranta, který nemá nebo pozbyl nárok na azyl. Fakticky na našem území bude zůstávat každý migrant včetně zločinců, který se k nám dostane. EU nás vede do chaosu👎🏻👎🏻👎🏻   

Drzost EU nezná mezí. Opět prosazuje přerozdělování imigrantů prostřednictvím kvót. EK se v příštích týdnech chystá předložit svůj návrh nové podoby unijní migrační a azylové strategie a podle zdroje ČTK jsou povinné kvóty v dokumentu obsaženy. Musíme je odmítnout! A CZEXIT👍🏻   

Kommentar zur Flüchtlingskrise: proč říci ne Evropě za ostnatým dráteml  [Kommentar zur Flüchtlingskrise: Warum ich kein Europa hinter Stacheldraht will  -
	Politik Ausland -
	Bild.de] 

Nový model tzv. evropské azylové politiky je jen nový pokus jak dál omezit státní suverenitu členských států Unie a ponížit nás. Návrh EK je neakceptovatelný, nařizuje nám se podřídit přerozdělování nebo platit za deportace migrantů do zemí původu.

## Sklearn Graph Creation

In [5]:
import torch
import numpy as np

def get_semantic_neighbors(search_pattern, data, k=100, max_num=2000):
    '''
    The function:
    (1) finds all tweets in the data that conform to the search pattern; and then
    (2) prints k tweets that are semanticaly closest to searched tweets, but do not conform 
    to the search pattern (ie are syntactically outside the search scope, but are semantically
    closest to the syntactic search result).
       (k tweets that are semantically closest = k tweets with the lowest sum of 
       cosine similarities to searched tweets).
    
    :param max_num: If more than max_num tweets are found in data in step (1), only max_num
    of such tweets (randomly chosen) are used for identifying semantically similar tweets in 
    step (2).
    Reason for the limitation: multiplying all tweets in the full dataset (186k) with too many 
    found tweets (> 10k for 'prezident') results in excessively long computation time).
    '''
    cont_indices_all = indices_containing(search_pattern, data)
    if len(cont_indices_all) > max_num:
        cont_indices_limit = random.sample(cont_indices_all, max_num)
    else:
        cont_indices_limit = cont_indices_all

    if len(cont_indices_all):
        tcf = top_closest_indices(cont_indices_limit, k=k, vectors=vecs, 
                                  data=all_lines, excl=cont_indices_all)
        return tcf
    else:
        print(f'No tweet with {search_pattern} found.')
        
import random
        
n1 = indices_containing('iverm', all_lines)
n2 = indices_containing('očkován', all_lines)
n3_all = list(set(range(len(all_lines))).difference(set(n1)).difference(set(n2)))
n3_sub = random.sample(n3_all, 3000)
    
v1 = vecs[n1,:]
v2 = vecs[n2,:]
v3_sub = vecs[n3_sub,:]
v3_all = vecs[n3_all,:]
print(v1.shape, v2.shape, v3_sub.shape, v3_all.shape)

torch.Size([106, 768]) torch.Size([574, 768]) torch.Size([3000, 768]) torch.Size([184482, 768])


In [29]:
print(all_lines[0])

{'id': 1422203548138450949, 'text': 'Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍'}


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np

xses_sub = torch.vstack([v1,v2,v3_sub])
ys_sub = torch.vstack([torch.full((len(v),1),i) for i,v in enumerate([v1,v2,v3_sub])])
ys_sub_np = ys_sub.squeeze().cpu().detach().numpy()

xses_all = torch.vstack([v1,v2,v3_all])
ys_all = torch.vstack([torch.full((len(v),1),i) for i,v in enumerate([v1,v2,v3_all])])
ys_all_np = ys_all.squeeze().cpu().detach().numpy()

import itertools
texts_sub = [all_lines[i]['text'][:100] for i in itertools.chain(n1,n2,n3_sub)]
texts_all = [all_lines[i]['text'][:100] for i in itertools.chain(n1,n2,n3_all)]

lda = LDA(n_components=2, solver='eigen')
lda.fit(xses_sub.cpu().detach().numpy(), ys_sub_np)
reslda = lda.transform(xses_all.cpu().detach().numpy())

fig = px.scatter(x=reslda[:,0], y=reslda[:,1], color=ys_all_np, hover_name=texts_all, height=600)
fig.update_layout(
    hoverlabel=dict(
        font_size=9,
    )
)
fig.show()

## Timeline

In [60]:
import pandas as pd
import pickle
import sys
from pathlib import Path

root_dir =  Path("/mnt/data/factcheck/twitter/people")
dfall = pd.read_pickle(Path(root_dir, "titles", "df_titles.pkl"))
pdalllines = pd.DataFrame(all_lines)
dfall = dfall.merge(pdalllines, on="id", how="left")

In [61]:
filtered = dfall[dfall.id.isin(set(twitter_ids))]

In [62]:
filtered[['name', 'date', 'text']]

name        date  \
93                           Alena Schillerová  2021-07-21   
141                          Alena Schillerová  2021-07-14   
841                          Alena Schillerová  2021-03-12   
982                          Alena Schillerová  2021-02-11   
1093                         Alena Schillerová  2021-01-15   
...                                        ...         ...   
198575                Markéta Pekarová Adamová  2021-08-04   
198634  Olga Richterová - poslankyně za Piráty  2021-08-07   
198697                          Ivan Bartoš ⚫⬛  2021-08-10   
198713                          Ivan Bartoš ⚫⬛  2021-08-04   
198800                          Miloš Vystrčil  2021-08-04   

                                                     text  
93      Další očkovací centrum bez registrace. To jsme...  
141     Souhlasím s, že sjednocení odvodů do jednoho i...  
841     Pokud jste dnes potkali někoho se srdcem na re...  
982     Dnes ve zvítězilo politikaření a předvolební k...  
1093    Mám dobrou zprávu pro lidi v insolvenci. Naše ...  
...                                                   ...  
198575  Upřímnou soustrast rodinám obětí střetu vlaků ...  
198634   I proto jsem mluvila o úřednících v dnešním r...  
198697  Viz. například kauza Krajská zdravotní a dosaz...  
198713  Otřesná tragédie při srážce vlaků u obce Milav...  
198800  Děkuji všem složkám IZS za rychlý zásah při ne...  

[1000 rows x 3 columns]

In [ ]:
from collections import Counter
import plotly.express as px
import textwrap

# adapted from JD
def make_timeline(dfall, twitter_ids, start_date="2019-10-01"):
    df_sel = dfall[dfall.id.isin(set(twitter_ids)) & (dfall.date >= start_date)].copy()
    df_sel["date_to"] = pd.to_datetime(df_sel.date) + pd.Timedelta(days=1)  # just for visualization
    df_sel["name_counts"] = df_sel.name.map(Counter(df_sel.name))
    df_sel["name_with_counts"] = df_sel.apply(lambda row: f"{row['name']} ({row.name_counts})", axis=1)
    df_sel.sort_values(["name_counts"], inplace=True, ascending=False)

    print(f"result shape: {df_sel.shape}")

    hover_name = [f"<b>{n}</b> ({d})<br>" + textwrap.fill(t, width=50).replace("\n", "<br>")
                  for _, (n, d, t) in df_sel[["name", "date", "text"]].iterrows()]

    fig = px.bar(df_sel, x="date", y=[1] * len(df_sel), color="name_with_counts",
                 labels={"date": "date", "y": "count"},
                 width=1200, height=500,
                 hover_name=hover_name)

    fig.update_layout(legend=dict(
        orientation="h",
        title_text=None,
        yanchor="top",
        y=-0.3,
        xanchor="left",
    ))

    fig.update_traces(marker_line_width=0.01)  # assures at least pixel even for many bars
    return fig

f = make_timeline(dfall=dfall, twitter_ids=twitter_ids)
f.show()